# 03. 基于GAT的节点分类(Cora-半监督）

In [1]:
## 1.  数据集

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='./dataset', name='Cora', transform=NormalizeFeatures())
data = dataset[0]  # 获取第一个图对象

print(f'数据集: {dataset}')
print(f'图数据: {data}')



/home/libing/miniconda3/envs/kk_gnn_wsl/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/libing/miniconda3/envs/kk_gnn_wsl/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/libing/miniconda3/envs/kk_gnn_wsl/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/libing/miniconda3/envs/kk_gnn_wsl/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/home/libing/miniconda3/envs/kk_gnn_wsl/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torc

数据集: Cora()
图数据: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [2]:
## GATConv

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv


class GAT(nn.Module):
    def __init__(self, out_channels=7):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels=1433, out_channels=64, heads=8, dropout=0.1, concat=False, negative_slope=0.2)
        self.conv2 = GATConv(in_channels=64, out_channels=out_channels, heads=1, dropout=0.1, concat=False, negative_slope=0.2)
        
        self.dp = nn.Dropout(p=0.5)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.dp(x)
        x = self.conv2(x, edge_index)
        return x


model = GAT()
print(model)

GAT(
  (conv1): GATConv(1433, 64, heads=8)
  (conv2): GATConv(64, 7, heads=1)
  (dp): Dropout(p=0.5, inplace=False)
)


In [3]:
from torchinfo import summary

summary(model, input_data=(data.x, data.edge_index))

Layer (type:depth-idx)                   Output Shape              Param #
GAT                                      [2708, 7]                 --
├─GATConv: 1-1                           [2708, 64]                1,088
│    └─Linear: 2-1                       [2708, 512]               733,696
│    └─SumAggregation: 2-2               [2708, 8, 64]             --
├─Dropout: 1-2                           [2708, 64]                --
├─GATConv: 1-3                           [2708, 7]                 21
│    └─Linear: 2-3                       [2708, 7]                 448
│    └─SumAggregation: 2-4               [2708, 1, 7]              --
Total params: 735,253
Trainable params: 735,253
Non-trainable params: 0
Total mult-adds (G): 1.99
Input size (MB): 15.69
Forward/backward pass size (MB): 11.24
Params size (MB): 2.94
Estimated Total Size (MB): 29.87

In [7]:
# 训练和评估

import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
data = data.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


for epoch in range(1000):
    # train
    model.train()
    optimizer.zero_grad()
    train_outputs = model(data.x, data.edge_index)
    loss = criterion(train_outputs[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    train_preds = train_outputs.argmax(dim=1)
    train_correct = train_preds[data.train_mask] == data.y[data.train_mask]
    train_acc = int(train_correct.sum()) / int(data.train_mask.sum())

    # test
    model.eval()
    test_outputs = model(data.x, data.edge_index)
    test_preds = test_outputs.argmax(dim=1)
    test_correct = test_preds[data.test_mask] == data.y[data.test_mask]
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch: 【{epoch+1}/{1000}】, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


Epoch: 【10/1000】, Loss: 0.0454, Train Acc: 1.0000, Test Acc: 0.7390
Epoch: 【20/1000】, Loss: 0.0606, Train Acc: 1.0000, Test Acc: 0.7490
Epoch: 【30/1000】, Loss: 0.0712, Train Acc: 1.0000, Test Acc: 0.7360
Epoch: 【40/1000】, Loss: 0.0515, Train Acc: 1.0000, Test Acc: 0.7380
Epoch: 【50/1000】, Loss: 0.0587, Train Acc: 0.9929, Test Acc: 0.7520
Epoch: 【60/1000】, Loss: 0.0430, Train Acc: 1.0000, Test Acc: 0.7500
Epoch: 【70/1000】, Loss: 0.0476, Train Acc: 1.0000, Test Acc: 0.7400
Epoch: 【80/1000】, Loss: 0.0546, Train Acc: 1.0000, Test Acc: 0.7290
Epoch: 【90/1000】, Loss: 0.0438, Train Acc: 1.0000, Test Acc: 0.7520
Epoch: 【100/1000】, Loss: 0.0546, Train Acc: 1.0000, Test Acc: 0.7480
Epoch: 【110/1000】, Loss: 0.0705, Train Acc: 1.0000, Test Acc: 0.7500
Epoch: 【120/1000】, Loss: 0.0423, Train Acc: 1.0000, Test Acc: 0.7450
Epoch: 【130/1000】, Loss: 0.0421, Train Acc: 1.0000, Test Acc: 0.7570
Epoch: 【140/1000】, Loss: 0.0703, Train Acc: 0.9929, Test Acc: 0.7540
Epoch: 【150/1000】, Loss: 0.0785, Train Acc: